## Fine tuning the Vision Transformer

In this notebook we use the pretrained ViT-L/16 model on the ImageNet-21k dataset which contains about 14 million images. The model will be finetuned on different datasets which are used for image classification tasks and then compared the performance to the baseline model.

We will use the pretrained weights on hugging face which are the same weights provided by the authors but translated to be used in Pytorch. The next few cells show the functions used for finetuning the **ViT-L/16** model on different datasets.

------------------------------------------------------------------------

We start by importing the required modules.

In [ ]:
import os
import json
import time
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import ViTForImageClassification
from torchvision import transforms, datasets, models

------------------------------------------------------------------------

Similar to what we did with the **ResNet** model, we create a dataloader that loads the same five datasets. The difference between the two functions is the size of the images in the dataloader as each model tolerates different input sizes for the images.

In [ ]:
# Create data loaders for transformer
def get_vit_loaders(dataset="imagenet", batch_size=64):
    """
    This loads the whole dataset into memory and returns train and test data to
    be used by the Vision Transformer
    @param dataset (string): dataset name to load
    @param batch_size (int): batch size for training and testing

    @returns dict() with train and test data loaders with keys `train_loader`, `test_loader`
    """
    # Normalization using channel means
    normalize_transform = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

    # Creating transform function
    train_transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), normalize_transform])

    # Test transformation function
    test_transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), normalize_transform])

    # Load the dataset from torchvision datasets
    if dataset == "imagenet":
        # Load ImageNet
        original_train_dataset = None
        train_loader = None
        test_transform = transforms.Compose([transforms.Resize((384,384)), transforms.ToTensor(), normalize_transform])
        original_test_dataset = datasets.ImageFolder(root=os.path.join('data', 'imagenet', 'val'), transform=test_transform)

    elif dataset == "cifar10":
        # Load CIFAR-10
        original_train_dataset = datasets.CIFAR10(root=os.path.join('data', 'cifar10_data'),
                                             train=True, transform=train_transform, download=True)
        original_test_dataset = datasets.CIFAR10(root=os.path.join('data', 'cifar10_data'),
                                             train=False, transform=test_transform, download=True)
    elif dataset == "cifar100":
        # Load CIFAR-100
        original_train_dataset = datasets.CIFAR100(root=os.path.join('data', 'cifar100_data'),
                                             train=True, transform=train_transform, download=True)
        original_test_dataset = datasets.CIFAR100(root=os.path.join('data', 'cifar100_data'),
                                             train=False, transform=test_transform, download=True)
    elif dataset == "oxford_pets":
        # Load Oxford-IIIT Pets
        original_train_dataset = datasets.OxfordIIITPet(root=os.path.join('data', 'oxford_iiit_pets_data'),
                                             split='trainval', transform=train_transform, download=True)
        original_test_dataset = datasets.OxfordIIITPet(root=os.path.join('data', 'oxford_iiit_pets_data'),
                                             split='test', transform=test_transform, download=True)
    elif dataset == "flowers_102":
        # Load Oxford Flowers-102
        original_train_dataset = datasets.Flowers102(root=os.path.join('data', 'oxford_flowers_102_data'),
                                             split='train', transform=train_transform, download=True)
        original_test_dataset = datasets.Flowers102(root=os.path.join('data', 'oxford_flowers_102_data'),
                                             split='test', transform=test_transform, download=True)
    else:
        # Raise an error if the dataset is not valid
        raise ValueError("Invalid dataset name. Please choose one of the following: imagenet, cifar10, cifar100, oxford_pets, flowers_102")

    # Creating data loaders
    loader_args = {
        "batch_size": batch_size,
    }

    if original_train_dataset is not None:
        train_loader = torch.utils.data.DataLoader(
            dataset=original_train_dataset,
            shuffle=True,
            **loader_args)

    test_loader = torch.utils.data.DataLoader(
        dataset=original_test_dataset,
        shuffle=True,
        **loader_args)

    return {"train_loader": train_loader,
            "test_loader": test_loader}

------------------------------------------------------------------------

-   `get_accuracy`: This function takes the model predictions and the true labels as inputs and returns the accuracy as a float value.
-   `evaluate_on_test`: This function takes the model, the loss criterion, the test dataloader, and the device as inputs and returns the test accuracy and loss as float values.

In [ ]:
# Function takes predictions and true values to return accuracies
def get_accuracy(logit, true_y):
    pred_y = torch.argmax(logit, dim=1)
    return (pred_y == true_y).float().mean()

# This Function is used to evaluate the model
def evaluate_on_test(model, test_loader, device):
    # Evaluate the model on all the test batches
    accuracies = []
    model.eval()
    for batch_idx, (data_x, data_y) in enumerate(test_loader):
        data_x = data_x.to(device)
        data_y = data_y.to(device)

        model_y = model(data_x)
        batch_accuracy = get_accuracy(model_y.logits, data_y)

        accuracies.append(batch_accuracy.item())

        if batch_idx%1000 == 0:
            print(f"Mean accuracy at batch: {batch_idx} is {np.mean(accuracies) * 100}")

    test_accuracy = np.mean(accuracies) * 100
    print(f"Test accuracy: {test_accuracy}")
    return test_accuracy

------------------------------------------------------------------------

The following function is used to train a vision transformer model on a given dataset and evaluate its performance on the train and test sets. The function takes the following arguments:

-   `loaders`: a dictionary of PyTorch dataloaders for the train, test, and validation sets.
-   `model_name`: a string to specify the name of the vision transformer model to use. The default is ‘google/vit-base-patch16-224-in21k’, which is a pre-trained model from Google that uses 16x16 patches and has 224 hidden units.
-   lr: a float to specify the learning rate for the optimizer. The default is 0.003.
-   `epochs`: an integer to specify the number of epochs to train the model. The default is 10.

The function returns two lists of floats, which are the train and test accuracies for each epoch.

In [ ]:
# Function to train the model and return train and test accuracies
def train_vit_model(loaders, title="", model_name='google/vit-large-patch16-224-in21k',
                         lr=0.003, epochs=10, random_seed=42, save=False):

    # Create experiment directory
    experiment_dir = os.path.join('experiments', title)

    # make experiment directory
    os.makedirs(experiment_dir, exist_ok=True)

    # Set the seed
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)

    # Check if GPU is available
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
        print("CUDA Recognized")
    else:
        device = torch.device('cpu')

    # Get num_classes
    num_classes = len(loaders["train_loader"].dataset.classes)

    # Load the pre-trained model
    model = ViTForImageClassification.from_pretrained(model_name, num_labels=num_classes)
    # Move the model to the device
    model = model.to(device)

    # Create the optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Get batch size
    batch_size = loaders["train_loader"].batch_size

    # Create the loss function
    criterion = torch.nn.CrossEntropyLoss()

    # Arrays to hold accuracies
    test_acc = [0]
    train_acc = [0]
    # Iterate over the number of epochs
    for epoch in range(1, epochs + 1):
        model.to(device);
        model.train()
        print(f"Epoch {epoch}")
        accuracies = []
        losses = []

        # Calculate loss and gradients for models on every training batch
        for batch_idx, (data_x, data_y) in enumerate(loaders["train_loader"]):
            data_x = data_x.to(device)
            data_y = data_y.to(device)

            optimizer.zero_grad()
            #model_y = model.classifier(model(data_x).pooler_output)
            model_y = model(data_x).logits
            loss = criterion(model_y, data_y)
            batch_accuracy = get_accuracy(model_y, data_y)

            # Perform back propagation
            loss.backward()
            optimizer.step()

            accuracies.append(batch_accuracy.item())
            losses.append(loss.item())

        # Store training accuracy for plotting
        train_loss = np.mean(losses)
        train_accuracy = np.mean(accuracies)
        train_acc.append(train_accuracy*100)

        print("Train accuracy: {} Train loss: {}".format(train_accuracy, train_loss))

        # Evaluate the model on all the test batches
        accuracies = []
        losses = []
        model.eval()
        # Move the model to CPU
        model.to("cpu")
        for batch_idx, (data_x, data_y) in enumerate(loaders["test_loader"]):
            # Move the data to CPU
            data_x = data_x.to("cpu")
            data_y = data_y.to("cpu")

            model_y = model(data_x).logits
            loss = criterion(model_y, data_y)
            batch_accuracy = get_accuracy(model_y, data_y)

            accuracies.append(batch_accuracy.item())
            losses.append(loss.item())

            # Break if there are more than 500 samples and not last iteration
            if (batch_idx+1)*batch_size > 500 and epoch < epochs:
                break

        # Store test accuracy for plotting
        test_loss = np.mean(losses)
        test_accuracy = np.mean(accuracies)
        test_acc.append(test_accuracy*100)
        print("Test accuracy: {} Test loss: {}".format(test_accuracy, test_loss))

    # Save the final model
    if save:
        torch.save({
            'model': model.state_dict()
        }, os.path.join(experiment_dir, f'Vit-L/16{title}.pt'))

    # return the accuracies
    return train_acc, test_acc

------------------------------------------------------------------------

The following function `plot_images_from_dataloader` takes a PyTorch dataloader as an input and plots 10 of the images from the first batch of data. The function also shows the labels of the images according to the classes attribute of the dataloader’s dataset

In [ ]:
# Define a function to plot 10 of the images
def plot_images_from_dataloader(dataloader):
    # Initialize empty tensors for images and labels
    images = torch.empty(0)
    labels = torch.empty(0, dtype=torch.long)

    # Loop until the images and labels have at least 10 elements
    while len(images) < 10:
        # Get the next batch of images and labels from the dataloader
        batch_images, batch_labels = next(iter(dataloader))
        # Concatenate the batch images and labels to the existing tensors
        images = torch.cat((images, batch_images), dim=0)
        labels = torch.cat((labels, batch_labels), dim=0)
    classes = dataloader.dataset.classes
    # Create a figure with 2 rows and 5 columns
    fig, axes = plt.subplots(2, 5, figsize=(10, 4))
    for i, ax in enumerate(axes.flat):
        image = images[i]
        label = classes[labels[i]]
        # Unnormalize the image
        image = image / 2 + 0.5
        image = image.numpy()
        # Transpose the image
        image = np.transpose(image, (1, 2, 0))
        # Plot the image on the axis
        ax.imshow(image)
        # Set title as label
        ax.set_title(label)
        # Turn off the axis ticks
        ax.set_xticks([])
        ax.set_yticks([])
    plt.show()

# Define a function to calculate runtime per dataset
def print_time(start_time, end_time):
    # Calculate the difference in seconds
    diff = end_time - start_time

    # Convert the difference to hours, minutes, and seconds
    hours, remainder = divmod(diff, 3600)
    minutes, seconds = divmod(remainder, 60)

    # Print the time in hours:minutes:seconds format
    print(f"Cell execution time: {int(hours)}:{int(minutes)}:{seconds}")

### ImageNet

The ImageNet dataset consists of **1000** object classes and contains **1,281,167** training images, **50,000** validation images and **100,000** test images. The images vary in resolution but it is common practice to train deep learning models on sub-sampled images of **256x256**pixels. This dataset is widely used for image classification and localization tasks and has been the benchmark for many state-of-the-art algorithms.

------------------------------------------------------------------------

The first thing we do is load the dataset into a PyTorch dataloader and plot a sample of 10 images to visualize it.

In [ ]:
# Plot some images from the ImageNet dataset
loader = get_vit_loaders(dataset="imagenet", batch_size=8)
plot_images_from_dataloader(loader["test_loader"])

------------------------------------------------------------------------

We load the model that was pretrained and fine-tuned on the **ImageNet-1k** which was published and evaluate the model directly.

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print("CUDA Recognized")
else:
    device = torch.device('cpu')

# Get num_classes
num_classes = len(loader["test_loader"].dataset.classes)

# Get the fine tuned model on the ImageNet dataset
model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-384')
# Move the model to the device
model = model.to(device)

------------------------------------------------------------------------

We evaluate the model on the train and test splits to get the train and test accuracies.

In [ ]:
start_time = time.time()
# Print the Performance of the Ready fine tuned model
test_acc_imagenet = evaluate_on_test(model, loader["test_loader"], device)
# Calculate and print cell execution time
end_time = time.time()
print_time(start_time, end_time)
# delete model to free gpu
del model
# Release unused memory
torch.cuda.empty_cache()

------------------------------------------------------------------------

We save the results for the **ImageNet** dataset in a dictionary to use later for comparing results.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/vit.json"):
    # Open the file in read mode
    with open("experiments/vit.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["imagenet"] = test_acc_imagenet                   

In [ ]:
# Save the outputs in a json file
with open("experiments/vit.json", "w") as f:
    json.dump(runs, f)

------------------------------------------------------------------------

### CIFAR-10

The CIFAR-10 dataset consists of **60,000 32x32** color images in **10** different classes. The 10 classes are airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. There are **6,000** images per class, with **5,000** for training and **1,000** for testing. It is a popular benchmark for image classification and deep learning research.

------------------------------------------------------------------------

First we load the dataset into dataloaders and plot a sample to visualize how the dataset looks like. **Note** that you should decide the `batch_size` that fits the GPU you are using.

In [ ]:
# Plot some images from the CIFAR-10 dataset
loader = get_vit_loaders(dataset="cifar10", batch_size=32)
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

We fine tune the pretrained vision transformer on the **CIFAR-10** dataset to get the train and test accuracies.

In [ ]:
start_time = time.time()
# Fine tune the model on CIFAR-10
train_acc_cifar10, test_acc_cifar10 = train_vit_model(loaders=loader)
# Calculate and print cell execution time
end_time = time.time()
print_time(start_time, end_time)

------------------------------------------------------------------------

We save the obtained results to use it later for the creating the results table.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/vit.json"):
    # Open the file in read mode
    with open("experiments/vit.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["cifar10"] = test_acc_cifar10[-1]

In [ ]:
# Save the outputs in a json file
with open("experiments/vit.json", "w") as f:
    json.dump(runs, f)

------------------------------------------------------------------------

### CIFAR-100

The CIFAR-100 dataset consists of **60,000 32x32** color images in **100** different classes. The 100 classes are grouped into 20 superclasses, such as aquatic mammals, flowers, insects, vehicles, etc. There are **600** images per class, with **500** for training and **100** for testing. It is also a commonly benchmark for image classification and deep learning research.

------------------------------------------------------------------------

We load the dataset into a PyTorch dataloader and plot the first 10 images from the train loader. **Note** that we need to make sure that the batch size is greater than or equal 10 to prevent any errors during plotting.

In [ ]:
# Plot some images from the CIFAR-100 dataset
loader = get_vit_loaders(dataset="cifar100", batch_size=32)
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

Now we are ready to fine tune the model on the **CIFAR100** dataset.

In [ ]:
start_time = time.time()
# Fine tune the model on CIFAR-100
train_acc_cifar100, test_acc_cifar100 = train_vit_model(loaders=loader)
# Calculate and print cell execution time
end_time = time.time()
print_time(start_time, end_time)

------------------------------------------------------------------------

We save the results in `runs` using the key `cifar100`.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/vit.json"):
    # Open the file in read mode
    with open("experiments/vit.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["cifar100"] = test_acc_cifar100[-1]

In [ ]:
# Save the outputs in a json file
with open("experiments/vit.json", "w") as f:
    json.dump(runs, f)

------------------------------------------------------------------------

### Oxford-IIIT Pets

The Oxford-IIIT Pets is a **37** category pet dataset with roughly **200** images for each class created by the Visual Geometry Group at Oxford. The images have large variations in scale, pose and lighting. All images have an associated ground truth annotation of breed, head ROI (region of interest), and pixel level trimap segmentation. The dataset is useful for fine-grained image classification and segmentation tasks.

------------------------------------------------------------------------

We start by loading the dataset and plot a random sample of 10 images from the train loader.

In [ ]:
# Plot some images from the Oxford-IIIT Pets dataset
loader = get_vit_loaders(dataset="oxford_pets", batch_size=32)
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

We fine-tune the vision transformer on the dataset for 10 epochs.

In [ ]:
start_time = time.time()
# Fine tune the model on Oxford-IIIT Pets
train_acc_oxford_pets, test_acc_oxford_pets = train_vit_model(loaders=loader)
# Calculate and print cell execution time
end_time = time.time()
print_time(start_time, end_time)

------------------------------------------------------------------------

We save the results in the `runs` dictionary as usual using the dataset name as key.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/vit.json"):
    # Open the file in read mode
    with open("experiments/vit.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["oxford_pets"] = test_acc_oxford_pets[-1]

In [ ]:
# Save the outputs in a json file
with open("experiments/vit.json", "w") as f:
    json.dump(runs, f)

------------------------------------------------------------------------

### Oxford Flowers-102

The Oxford Flowers-102 dataset consists of **102** flower categories commonly occurring in the United Kingdom. Each class consists of between **40 and 258** images. The images have large scale, pose and light variations. In addition, there are categories that have large variations within the category and several very similar categories. The dataset also provides image labels, segmentations, and distances based on shape and color features.

------------------------------------------------------------------------

For this dataset we need to define the classes array as it is not defined in the PyTorch dataset. We load the dataset into PyTorch data loader and plot the first 10 images.

In [ ]:
# Plot some images from the Oxford Flowers-102 Pets dataset
loader = get_vit_loaders(dataset="flowers_102", batch_size=32)

# We initialize the flowers names as they are not on Pytorch (used for plotting)
flower_classes = ['pink primrose', 'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',
 'english marigold', 'tiger lily', 'moon orchid', 'bird of paradise', 'monkshood', 'globe thistle',
 'snapdragon', "colt's foot", 'king protea', 'spear thistle', 'yellow iris', 'globe-flower', 'purple coneflower',
 'peruvian lily', 'balloon flower', 'giant white arum lily', 'fire lily', 'pincushion flower', 'fritillary',
 'red ginger', 'grape hyacinth', 'corn poppy', 'prince of wales feathers', 'stemless gentian', 'artichoke',
 'sweet william', 'carnation', 'garden phlox', 'love in the mist', 'mexican aster', 'alpine sea holly',
 'ruby-lipped cattleya', 'cape flower', 'great masterwort', 'siam tulip', 'lenten rose', 'barbeton daisy',
 'daffodil', 'sword lily', 'poinsettia', 'bolero deep blue', 'wallflower', 'marigold', 'buttercup', 'oxeye daisy',
 'common dandelion', 'petunia', 'wild pansy', 'primula', 'sunflower', 'pelargonium', 'bishop of llandaff', 'gaura',
 'geranium', 'orange dahlia', 'pink-yellow dahlia', 'cautleya spicata', 'japanese anemone', 'black-eyed susan',
 'silverbush', 'californian poppy', 'osteospermum', 'spring crocus', 'bearded iris', 'windflower', 'tree poppy',
 'gazania', 'azalea', 'water lily', 'rose', 'thorn apple', 'morning glory', 'passion flower', 'lotus', 'toad lily',
 'anthurium', 'frangipani', 'clematis', 'hibiscus', 'columbine', 'desert-rose', 'tree mallow', 'magnolia',
 'cyclamen', 'watercress', 'canna lily', 'hippeastrum', 'bee balm', 'ball moss', 'foxglove', 'bougainvillea',
 'camellia', 'mallow', 'mexican petunia', 'bromelia', 'blanket flower', 'trumpet creeper', 'blackberry lily']

# Save the Class names in the dataset
loader["train_loader"].dataset.classes = flower_classes
# Plot dataset
plot_images_from_dataloader(loader["train_loader"])

------------------------------------------------------------------------

We fine-tune the model using the `train_vit_model` function and using the `Flowers102` dataset loader as the input arguement.

In [ ]:
start_time = time.time()
# Fine tune the model on Oxford Flowers-102
train_acc_oxford_flowers, test_acc_oxford_flowers = train_vit_model(loaders=loader, epochs=14)
# Calculate and print cell execution time
end_time = time.time()
print_time(start_time, end_time)

------------------------------------------------------------------------

Now we save the fine-tuning results in the `runs` dictionary.

In [ ]:
# Create dictionary runs
runs = {}

# Check if the file exists
if os.path.exists("experiments/vit.json"):
    # Open the file in read mode
    with open("experiments/vit.json", "r") as f:
        # Load the data from the file to runs
        runs = json.load(f)

# Add the results to a dictionary
runs["oxford_flowers"] = test_acc_oxford_flowers[-1]

In [ ]:
# Save the outputs in a json file
with open("experiments/vit.json", "w") as f:
    json.dump(runs, f)

------------------------------------------------------------------------

Now that we are done with both the ResNet and the ViT model, we can create the results table back at the Experiments notebook.